In [352]:
import os
import pickle
import numpy as np
import glob
import copy
from transformers import OpenAIGPTModel, OpenAIGPTLMHeadModel, OpenAIGPTDoubleHeadsModel, OpenAIGPTTokenizer
from itertools import chain
from ast import literal_eval
from itertools import zip_longest
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader 

In [64]:
%load_ext autoreload
%autoreload 2

### Data Preprocessing

In [323]:
a = [1,2,3]
it = iter(a)
testt = [it]*3

In [325]:
a[-(2*10+1):]

[1, 2, 3]

In [302]:
test_tok = tokenizer.tokenize('I agree that I am moderately bearable!')
test_tok

['i</w>',
 'agree</w>',
 'that</w>',
 'i</w>',
 'am</w>',
 'moderately</w>',
 'bearable</w>',
 '!</w>']

In [256]:
tokenizer.convert_tokens_to_ids(test_tok) # all tokens must be lowercase 

[249, 3798, 525, 249, 1048, 29121, 26055, 267]

### Severe TODO 
H Parser είναι πιο χρονοβόρα από την extract pairs
refactor το που γίνεται τι ώστε κάθε συνάρτηση να έχει ένα ρόλο 
το tokenization δεν κολλάει πολύ στην parser.

In [338]:
workspace = 'C:\\Users\\nikmand\\nikmand\\ncsr-chatbot\\'  # os.getcwd()

# TODO new function for the tokenization process

def parser(datafolder='metalwoz-v1\\dialoguesTest\\'): # rename to parser
    """
    Function that reads files, keeps only 'turns' from each entry and tokenizes them

    :param datafolder: path to the folder that contains the files
    :return: a list that contains dialogs, each dialog is a list of lists where each of them represents the ids of a phrase 
    """
    dialogs = []
    files = list(glob.glob(workspace + datafolder + "*.txt"))
    for file in files:
        with open(file) as f:
            for line in f.readlines():
                dialog = literal_eval(line)['turns'][1:] # keep only turns without the first sentence
                dialog = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(phrase)) for phrase in dialog] # to lowercase is performed by the tokenizer 
                dialogs.append(dialog)      
    return dialogs        

In [339]:
dialogs = parser()
print(dialogs[0])

[[249, 1048, 6702], [498, 1385, 512, 640], [488, 249, 1074, 12361, 15354, 504, 481, 3361], [249, 2518, 512, 1074, 246, 5358, 500, 481, 3361], [668, 5611, 239, 249, 1074, 688, 504, 2306], [525, 256, 252, 246, 1875, 4778], [912, 249, 1048, 246, 16219, 267], [249, 2310, 256, 241, 2153, 485, 699, 512], [488, 249, 15864, 512], [249, 1048, 481, 4721, 2351]]


In [307]:
def extract_pairs(dialogs, cache_file='cache_folder\\pairs.txt'):
    """
    Function that creates pairs of input, output from dialogs, each dialogs corresponds now to many pairs.
    
    :param: a list with all the dialogs 
    :return a list whose elements are pairs of input, output  
    """
    try: 
        with open(cache_file, "rb") as f:
            print("Cache file found loading content.")
            pairs = pickle.load(f)
            return pairs
    except: # cache file not created yet
        print("Cache file not found. Start processing.")
        pairs = [] 
        for dialog in dialogs:
            t_dict = {'input': []}
            if len(dialog) % 2 != 0: # discard the last phrase if it was said by the user
                dialog = dialog[:-1]
            dialog_it = iter(dialog)
            for i_phrase, o_phrase in zip_longest(dialog_it, dialog_it): # process phrases two by two
                try:
                    t_dict["input"].append(t_dict["output"])
                except:
                    pass
                t_dict["input"].append(i_phrase) # history
                t_dict["output"] = o_phrase
                pairs.append(t_dict)
                t_dict = copy.deepcopy(t_dict) # so future changes address only the new dict
        with open(cache_file, "wb") as f:
            pickle.dump(pairs, f)
        return pairs

In [340]:
pairs = extract_pairs(dialogs) # list of dictionaries of two keys

Cache file not found. Start processing.


In [341]:
len(pairs)

17496

In [233]:
tokenizer.max_len

512

In [326]:
dataset_cache = "dataset_cache"

### TODO να σώζεται σε αρχείο στην πιο κατάλληλη μορφή. Να δούμε αν βολεύει Pandas ή κάτι άλλο 

We instantiate a gpt pytorch model with pre-trained weights on language modelling task.

In [6]:
model_single = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
print(model_single)
model = OpenAIGPTDoubleHeadsModel.from_pretrained('openai-gpt')
print(model)
model_raw = OpenAIGPTModel.from_pretrained('openai-gpt')
print(model_raw) # without the last linear layer

100%|███████████████████████████████████████████████████████████████████████| 458495/458495 [00:00<00:00, 517807.43B/s]
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [ ]:
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

### Tokenizer

A helper class used to interact with the vocabulary in which our model has been pre-trained.

In [49]:
print("Our language model have been pre-trained with a vocabulary of {} words.".format(tokenizer.vocab_size))

Our language model have been pre-trained with a vocabulary of 40478 words.


In [30]:
# We will use 5 special tokens:
# - <bos> to indicate the start of the sequence
# - <eos> to indicate the end of the sequence
# - <speaker1> to indicate the beginning and the tokens of an utterance from the user
# - <speaker2> to indicate the beginning and the tokens of an utterance from the bot
# - <pad> as a padding token to build batches of sequences

SPECIAL_TOKENS = ["<bos>", "<eos>", "<speaker1>", "<speaker2>", "<pad>"]
ATTR_TO_SPECIAL_TOKEN = {'bos_token': '<bos>', 'eos_token': '<eos>', 'pad_token': '<pad>',
                         'additional_special_tokens': ('<speaker1>', '<speaker2>')}

bos, eos, speaker1, speaker2 = "<bos>", "<eos>", "<speaker1>", "<speaker2>"

MODEL_INPUTS = ["input_ids", "mc_token_ids", "lm_labels", "mc_labels", "token_type_ids"]
PADDED_INPUTS = ["input_ids", "lm_labels", "token_type_ids"]

In [15]:
def add_special_tokens_(model, tokenizer):
    """ Add special tokens to the tokenizer and the model if they have not already been added. """
    orig_num_tokens = len(tokenizer.encoder)
    num_added_tokens = tokenizer.add_special_tokens(ATTR_TO_SPECIAL_TOKEN) # doesn't add if they are already there
    if num_added_tokens > 0:
        model.resize_token_embeddings(new_num_tokens=orig_num_tokens + num_added_tokens)

In [20]:
def pad_dataset(dataset, padding=0):
    """ Pad the dataset. This could be optimized by defining a Dataset class and padding at the batch level, but this is simpler. """
    max_l = max(len(x) for x in dataset["input_ids"])
    for name in PADDED_INPUTS:
        dataset[name] = [x + [padding if name != "lm_labels" else -1] * (max_l - len(x)) for x in dataset[name]]
    return dataset

In [22]:
def build_inputs(persona, history, reply):
    # Build our sequence by adding delimiters and concatenating
    sequence = [[bos] + list(chain(*persona))] + history + [reply + [eos]]
    sequence = [sequence[0]] + [ [speaker2 if (len(sequence)-i) % 2 else speaker1] + s
                                for i, s in enumerate(sequence[1:])]
    # Build our word, segments and position inputs from the sequence
    words = list(chain(*sequence))                          # word tokens
    segments = [speaker2 if i % 2 else speaker1             # segment tokens
                for i, s in enumerate(sequence) for _ in s]
    position = list(range(len(words)))                      # position tokens
    return words, segments, position, sequence

In [18]:
add_special_tokens_(model, tokenizer)

In [41]:
# debug cell

persona = [["i", "like", "playing", "football", "."],
           ["i", "am", "from", "NYC", "."]]
history = [["hello", "how", "are", "you", "?"],
           ["i", "am", "fine", "thanks", "."]]
reply = ["great", "to", "hear"]

sequence = [["<bos>"] + list(chain(*persona))] + history +  [reply + ["<eos>"]]

sequence = [sequence[0]] + [ [speaker2 if (len(sequence)-i) % 2 else speaker1] + s for i, s in enumerate(sequence[1:])]

print(sequence)
print(list(chain(*sequence))   )

In [46]:
[speaker2 if i % 2 else speaker1 for i, s in enumerate(sequence) for _ in s]

['<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker2>',
 '<speaker2>',
 '<speaker2>',
 '<speaker2>',
 '<speaker2>',
 '<speaker2>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker1>',
 '<speaker2>',
 '<speaker2>',
 '<speaker2>',
 '<speaker2>',
 '<speaker2>']

In [269]:
def build_input_from_segments_or(persona, history, reply, tokenizer, lm_labels=False, with_eos=True):
    """ Build a sequence of input from 3 segments: persona, history and last reply. """
    bos, eos, speaker1, speaker2 = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[:-1])
    sequence = [[bos] + list(chain(*persona))] + history + [reply + ([eos] if with_eos else [])]  # chain: expands the lists  
    sequence = [sequence[0]] + [[speaker2 if (len(sequence)-i) % 2 else speaker1] + s for i, s in enumerate(sequence[1:])]
    instance = {}
    instance["input_ids"] = list(chain(*sequence)) # words
    instance["token_type_ids"] = [speaker2 if i % 2 else speaker1 for i, s in enumerate(sequence) for _ in s]
    instance["mc_token_ids"] = len(instance["input_ids"]) - 1
    instance["lm_labels"] = [-1] * len(instance["input_ids"])
    if lm_labels:
        instance["lm_labels"] = ([-1] * sum(len(s) for s in sequence[:-1])) + [-1] + sequence[-1][1:]
    return instance

Εμείς έχουμε πάντα περιττού πλήθους history που αρχίζει και τελειώνει με speaker1 και reply που το λέει ο speaker2

Για το input_ids: Η λογική είναι αναθέτει τον speaker2 κάθε φορά που μένει άρτιο πλήθος από διαλόγους(περιττό συνολικά μαζί με sos, βλέπε συνθήκη). Εμάς όλες μας οι λίστες έχουν άρτιο πλήθος οπότε θα ξεκινήσει με speaker2 ενώ θέλουμε speaker1. Τα επιμέρους αποτελεσματα όμως είναι συμβατά μεταξύ τους.

Στο input_ids το i στο iter παίρνει τιμή i = seq_len - 2 (αφού ξεκινήσαμε από το δεύτερο στοιχείο το iteration)

Για το token_type_ids: για κάθε μία λίστα κάνουμε iterate στα στοιχεία της, αν η θέση της λίστας είναι άρτια παίρνει speaker1 αλλιώς speaker2
Στο token_type_ids: επειδή το πλήθος είναι περιττό με την προσθήκη του sos θα αλλάξει η σειρά και η πρώτη πρόταση θα πάει speaker2 και το reply speaker1

Καταρχάς η αντιστοιχία που δίνουν οι ίδιοι στο δικό τους δεν ταιριάζει με αυτό που είχαμε σκεφτεί 
Κατά δεύτερο πρέπει να δούμε που θα μπει αν θα μπει το sos, αυτό μας δημιουργεί πρόβλημα αυτή τη στιγμή. Θα μπει μετά το tag του speaker ? 

είτε θα μπει μόνο του πριν τον speaker
σε αυτή την περίπτωση θα πρέπει να παίρνει το tag του speaker1 στα tokens αυτό δε συμβαίνει τώρα και μας μπερδεύει τη σειρά 

In [289]:
history = [[249, 1048, 6702], [498, 1385, 512, 640], [488, 249, 1074, 12361, 15354, 504, 481, 3361]]
reply =  [249, 2518, 512, 1074, 246, 5358, 500, 481, 3361]

instance = build_input_from_segments(history, reply, tokenizer)
instance_or = build_input_from_segments_or(persona, history, reply, tokenizer, lm_labels=True)

5
[[40478], [40481, 249, 1048, 6702], [40482, 498, 1385, 512, 640], [40481, 488, 249, 1074, 12361, 15354, 504, 481, 3361], [40482, 249, 2518, 512, 1074, 246, 5358, 500, 481, 3361, 40479]]


In [263]:
print(tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[:-1]))

[40478, 40479, 40481, 40482]


In [274]:
instance_or["input_ids"]

[40478,
 'i',
 'like',
 'playing',
 'football',
 '.',
 'i',
 'am',
 'from',
 'NYC',
 '.',
 40482,
 249,
 1048,
 6702,
 40481,
 498,
 1385,
 512,
 640,
 40482,
 488,
 249,
 1074,
 12361,
 15354,
 504,
 481,
 3361,
 40481,
 249,
 2518,
 512,
 1074,
 246,
 5358,
 500,
 481,
 3361,
 40479]

In [273]:
instance_or["token_type_ids"]

[40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40482,
 40482,
 40482,
 40482,
 40481,
 40481,
 40481,
 40481,
 40481,
 40482,
 40482,
 40482,
 40482,
 40482,
 40482,
 40482,
 40482,
 40482,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481]

In [290]:
instance["input_ids"] 

[40478,
 40481,
 249,
 1048,
 6702,
 40482,
 498,
 1385,
 512,
 640,
 40481,
 488,
 249,
 1074,
 12361,
 15354,
 504,
 481,
 3361,
 40482,
 249,
 2518,
 512,
 1074,
 246,
 5358,
 500,
 481,
 3361,
 40479]

In [291]:
instance["token_type_ids"] 

[40481,
 40481,
 40481,
 40481,
 40481,
 40482,
 40482,
 40482,
 40482,
 40482,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40481,
 40482,
 40482,
 40482,
 40482,
 40482,
 40482,
 40482,
 40482,
 40482,
 40482,
 40482]

In [54]:
instance = build_input_from_segments(persona, history, reply, tokenizer)

### ασυμβίβαστα μεταξύ τους στο input_ids βγαίνει ότι το reply το είπε ο speaker1, ενώ στο token_type_ids ότι το είπε ο speaker2 (μάλλον για τον κώδικα του medium μόνο)

In [129]:
lm_targets = ([-1] * sum(len(s) for s in sequence[:-1])) \
             + [-1] + tokenizer.convert_tokens_to_ids(sequence[-1][1:])

lm_targets # στα labels tou language model έχουν τιμές μόνο τα tokens του reply.
lm_distractor = [-1] * len(instance["input_ids"])
lm_distractor

Τι ακριβώς θα δούμε με το validation.

In [342]:
pairs_array = np.array(pairs)

In [350]:
pairs_train_l = pairs[:int(len(pairs)*0.8)]
pairs_eval_l = pairs[int(len(pairs)*0.8):]
print(len(pairs_train_l), len(pairs_eval_l))

13996 3500


### Για διαχωρισμό σε train test
υπάρχει κάτι που να δημιουργεί πρόβλημα;
μπορεί να μας ενοχλεί ότι ζεύγη που έρχονται από διαφορετικούς διαλόγους θα χωριστούν; μας ενοχλεί αν δεν μπαίνουν με τη σειρά;
αν δεν κάνουμε τυχαίο split κάποια domains δε θα εμφανίζονται στο train set

In [343]:
pairs_train, pairs_eval = train_test_split(pairs_array, test_size=0.2)      

In [345]:
print(pairs_train.shape, pairs_eval.shape)
print(pairs_train[0])
print("wwwwwww")
print(pairs_eval[0]['input'])

(13996,) (3500,)
{'input': [[4895, 20721], [773], [15670, 589, 4895, 562, 1115, 2106]], 'output': [512, 604, 1233, 284, 17177, 562, 1115, 2106, 240, 587, 512, 823, 485, 15670, 688, 589, 257]}
wwwwwww
[[249, 1074, 12361, 15354, 504, 481, 3361, 239, 966, 803, 1150, 239, 512, 886, 720, 257], [881, 240, 249, 256, 527, 886, 720, 267], [1424, 267, 616, 544, 1424, 1499, 770, 257], [507, 544, 246, 1424, 1499, 645, 512, 635, 1074, 15354, 504, 481, 3361, 239, 616, 1532, 247, 256, 241, 1743], [249, 759, 587, 507, 249, 1048, 481, 4735, 1559, 257], [498, 1385, 512, 256, 716, 481, 4735, 240, 568, 507, 256, 252, 843, 595, 1743], [249, 256, 258, 1359, 1662, 556, 25099, 25487, 488, 4864, 528, 36, 2616, 239, 606, 812, 925, 616, 2116, 239, 1572, 3798, 239], [249, 3798, 556, 512, 525, 512, 812, 925, 616, 2116], [2190, 267, 249, 668, 1388, 485, 1344, 616, 617, 1184, 239, 620, 1311, 1473, 980, 1014, 510, 525, 249, 759, 256, 241, 239]]


Θέλω στο μοντέλο μου να δίνω τρία inputs όπως το παράγει η συνάρτηση build, συνεπώς αυτό θέλω να μου γυρίζει η συνάρτηση get item 

In [364]:
class DialogDataset(Dataset):

    def __init__(self, pairs):
        self.dataset = self.create_segments(pairs)
        # TODO padding

    def __len__(self):
        return len(self.dataset)
    
    def create_segments(self, pairs):
        dataset = []
        for pair in pairs:
            instance = build_input_from_segments(pair['input'], pair['output'], tokenizer)
            dataset.append(instance)
        return dataset
    
    def pad_sequense(self, dataset):
        # TODO create mask for each element
        pass

    def __getitem__(self, index):
#         index_ids = self.index_ids[index]
#         category_ids = self.category[index]
#         labels_ids = self.labels[index]
        sample = self.dataset[index]
        return (sample['index_ids'], sample['category_ids'], sample['labels_ids'])

In [361]:
def build_input_from_segments(history, reply, tokenizer, with_eos=True):
    """ Build a sequence of input from 2 segments:  history and last reply. """
    bos, eos, speaker1, speaker2 = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[:-1])
    sequence = [[bos]] + history + [reply + ([eos] if with_eos else [])]
    seq_len = len(sequence) # sequence λίστα από λίστες
    # print(seq_len)
    sequence = [sequence[0]] + [[speaker2 if (seq_len-i) % 2 != 1 else speaker1] + s for i, s in enumerate(sequence[1:])]
    # print(sequence)
    instance = {}
    instance["input_ids"] = list(chain(*sequence)) # words
    instance["token_type_ids"] = [speaker1] + [speaker2 if i % 2 else speaker1 for i, s in enumerate(sequence[1:]) for _ in s] # for each word
    # TODO mask
    instance["lm_labels"] = ([-1] * sum(len(s) for s in sequence[:-1])) + [-1] + sequence[-1][1:]
    return instance

In [365]:
training_set = DialogDataset(pairs_train) 
validation_set = DialogDataset(pairs_eval)

In [367]:
BATCH_SIZE = 32 

dataloader_train = DataLoader(training_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) # has shuffle any sideffects here?

dataloader_valid = DataLoader(validation_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

### Training procedure

In [ ]:
epochs = 2  
min_loss, max_patience, cur_patience = np.inf, 10, 0
save_file = "chatbot.pkl"
use_cuda = False

# loss_function = nn. check it την διαλέγει μόνο του?
# αν δοθεί το labels αρχικοποιεί και χρησιμοποιεί εσωτερικά το crossEntropyLoss
# να πούμε αναλυτικά τι κάνει σε πρώτη φάση, αν δε το γράψουμε χεράτα.
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.001)
for epoch in range(epochs):
    # aux staff here
    for i_batch, (input_ids, category_ids, labels) in enumerate(dataloader_train):
        outputs = model(input_ids=input_ids, token_type_ids=category_ids, labels=labels)
        loss, logits = outputs[:2]
        
        # στο loss που παίρνω μπορώ να κάνω backward απ' ότι καταλαβαίνω
        
        

### Interaction with the bot - Inference